# Seattle Public Library (SPL) data analysis (inventory)

This notebook performs some exploratory analysis on the [Seattle Public Library Checkout Records dataset](https://www.kaggle.com/seattle-public-library/seattle-library-checkout-records) found on Kaggle.

This notebook will be exploring the inventory file that contains a list of books at the SPL.

## Set up and load data

In [1]:
import os

import findspark
findspark.init()

from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, FloatType, StringType, StructField, StructType
from pyspark_dist_explore import hist

%matplotlib inline

spark = SparkSession.builder.appName("ExploreSplInventory").getOrCreate()

In [2]:
load_dotenv()

SPL_INVENTORY_PATH = os.getenv("SPL_INVENTORY_PATH")

In [3]:
spl_df = spark.read.option("header", "true").csv(SPL_INVENTORY_PATH)

# Add a column that stores report date as a datetime object
spl_df = spl_df.withColumn(
    "ReportDateTS", 
    F.to_timestamp(spl_df.ReportDate, "MM/dd/yyyy"),
)

# Helpers
This section contains some helper functions that will be reused when analyzing each column.

In [4]:
def get_basic_counts(df, col):
    df.agg(
        F.count(col),
        F.countDistinct(col),
    ).show()
    
    
def check_nulls(df, null_col, id_coll):
    df.where(null_col.isNull()).agg(
        F.count(id_coll),
        F.countDistinct(id_coll),
    ).show()

    
def check_empty_strings(df, col):
    df.where(col == "").agg(
        F.count(col),
        F.countDistinct(col),
    ).show()
   

def check_lengths(df, col):
    (
        df
        .select(col, F.length(col))
        .sort(F.length(col).asc())
        .show(5) 
    )
    (
        df
        .select(col, F.length(col))
        .sort(F.length(col).desc())
        .show(5) 
    )
    
    
def basic_stats(df, col):
    df.agg(
        F.count(col),
        F.avg(col),        
        F.min(col),
        F.max(col),
    ).show() 
    
    
def plot_hist(df, bins, title=None):
    fig, ax = plt.subplots()
    hist(ax, df, bins=bins, color=["blue"])
    if title:
        ax.set_title(title)
        _ = ax.legend()   

## Inspect table

In [5]:
spl_df.show(5)

+-------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+--------------+------------+------------+----------+---------+-------------------+
| BibNum|               Title|              Author|                ISBN|PublicationYear|           Publisher|            Subjects|ItemType|ItemCollection|FloatingItem|ItemLocation|ReportDate|ItemCount|       ReportDateTS|
+-------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+--------+--------------+------------+------------+----------+---------+-------------------+
|3011076|A tale of two fri...|       O'Ryan, Ellie|1481425730, 14814...|          2014.|    Simon Spotlight,|Musicians Fiction...|    jcbk|         ncrdr|    Floating|         qna|09/01/2017|        1|2017-09-01 00:00:00|
|2248846|Naruto. Vol. 1, U...|Kishimoto, Masash...|          1569319006|   2003, c1999.|                Viz,|Nin

## BibNum

- What is the reason for so few distinct BibNums?
- Is BibNum sequentially?

In [6]:
get_basic_counts(spl_df, spl_df.BibNum)
check_nulls(spl_df, spl_df.BibNum, spl_df.Title)
check_empty_strings(spl_df, spl_df.BibNum)
check_lengths(spl_df, spl_df.BibNum)

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|      2687149|                584391|
+-------------+----------------------+

+------------+---------------------+
|count(Title)|count(DISTINCT Title)|
+------------+---------------------+
|           0|                    0|
+------------+---------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|            0|                     0|
+-------------+----------------------+

+------+--------------+
|BibNum|length(BibNum)|
+------+--------------+
|     4|             1|
|     7|             1|
|     7|             1|
|     4|             1|
|    91|             2|
+------+--------------+
only showing top 5 rows

+-------+--------------+
| BibNum|length(BibNum)|
+-------+--------------+
|3202727|             7|
|3225705|             7|
|3053201|             7|
|2512885|             7|
|28091

## Title

- Interestingly a lot of duplicate titles. Also appears to be close to the number of distinct BibNums.
- Some titles have nulls

In [7]:
get_basic_counts(spl_df, spl_df.Title)
check_nulls(spl_df, spl_df.Title, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.Title)
check_lengths(spl_df, spl_df.Title)

+------------+---------------------+
|count(Title)|count(DISTINCT Title)|
+------------+---------------------+
|     2672825|               567617|
+------------+---------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|        14324|                  7158|
+-------------+----------------------+

+------------+---------------------+
|count(Title)|count(DISTINCT Title)|
+------------+---------------------+
|           0|                    0|
+------------+---------------------+

+-----+-------------+
|Title|length(Title)|
+-----+-------------+
| null|         null|
| null|         null|
| null|         null|
| null|         null|
| null|         null|
+-----+-------------+
only showing top 5 rows

+--------------------+-------------+
|               Title|length(Title)|
+--------------------+-------------+
|Nation's forests ...|         1228|
|Nation's forests ...|         1228|
|Nominations of Jo...|  

## Author

- Some null authors 
- Mostly distinct authors (messy field, may need to clean up)

In [8]:
get_basic_counts(spl_df, spl_df.Author)
check_nulls(spl_df, spl_df.Author, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.Author)
check_lengths(spl_df, spl_df.Author)

+-------------+----------------------+
|count(Author)|count(DISTINCT Author)|
+-------------+----------------------+
|      2260911|                218757|
+-------------+----------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|       426238|                 92782|
+-------------+----------------------+

+-------------+----------------------+
|count(Author)|count(DISTINCT Author)|
+-------------+----------------------+
|            0|                     0|
+-------------+----------------------+

+------+--------------+
|Author|length(Author)|
+------+--------------+
|  null|          null|
|  null|          null|
|  null|          null|
|  null|          null|
|  null|          null|
+------+--------------+
only showing top 5 rows

+--------------------+--------------+
|              Author|length(Author)|
+--------------------+--------------+
| famous scout and...|           217|
| famous scout and.

# ISBN

- Some ISBNs are NULL which will make it harder to link up a book
- Some very long ISBN lists

In [9]:
get_basic_counts(spl_df, spl_df.ISBN)
check_nulls(spl_df, spl_df.ISBN, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.ISBN)
check_lengths(spl_df, spl_df.ISBN)

+-----------+--------------------+
|count(ISBN)|count(DISTINCT ISBN)|
+-----------+--------------------+
|    2099924|              397501|
+-----------+--------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|       587225|                186210|
+-------------+----------------------+

+-----------+--------------------+
|count(ISBN)|count(DISTINCT ISBN)|
+-----------+--------------------+
|          0|                   0|
+-----------+--------------------+

+----+------------+
|ISBN|length(ISBN)|
+----+------------+
|null|        null|
|null|        null|
|null|        null|
|null|        null|
|null|        null|
+----+------------+
only showing top 5 rows

+--------------------+------------+
|                ISBN|length(ISBN)|
+--------------------+------------+
|0788403923, 07884...|        1186|
|0788403923, 07884...|        1186|
|0691015856, 06910...|        1159|
|0691015856, 06910...|        1

## Publication Year

- Has NULLs
- Not very clean

In [10]:
get_basic_counts(spl_df, spl_df.PublicationYear)
check_nulls(spl_df, spl_df.PublicationYear, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.PublicationYear)
check_lengths(spl_df, spl_df.PublicationYear)

+----------------------+-------------------------------+
|count(PublicationYear)|count(DISTINCT PublicationYear)|
+----------------------+-------------------------------+
|               2654773|                          16225|
+----------------------+-------------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|        32376|                 14841|
+-------------+----------------------+

+----------------------+-------------------------------+
|count(PublicationYear)|count(DISTINCT PublicationYear)|
+----------------------+-------------------------------+
|                     0|                              0|
+----------------------+-------------------------------+

+---------------+-----------------------+
|PublicationYear|length(PublicationYear)|
+---------------+-----------------------+
|           null|                   null|
|           null|                   null|
|           null|     

## Publisher

- Has NULLs

In [11]:
get_basic_counts(spl_df, spl_df.Publisher)
check_nulls(spl_df, spl_df.Publisher, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.Publisher)
check_lengths(spl_df, spl_df.Publisher)

+----------------+-------------------------+
|count(Publisher)|count(DISTINCT Publisher)|
+----------------+-------------------------+
|         2649210|                    96894|
+----------------+-------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|        37939|                 18362|
+-------------+----------------------+

+----------------+-------------------------+
|count(Publisher)|count(DISTINCT Publisher)|
+----------------+-------------------------+
|               0|                        0|
+----------------+-------------------------+

+---------+-----------------+
|Publisher|length(Publisher)|
+---------+-----------------+
|     null|             null|
|     null|             null|
|     null|             null|
|     null|             null|
|     null|             null|
+---------+-----------------+
only showing top 5 rows

+--------------------+-----------------+
|           Publ

### Check possibility of duplicates

- There may be duplicate publishers

In [19]:
(
    spl_df
    .select(spl_df.Publisher)
    .distinct()
    .filter(F.lower(spl_df.Publisher).startswith("scho"))
    .sort(spl_df.Publisher.asc()).show()
)

+--------------------+
|           Publisher|
+--------------------+
|Schocken : Distri...|
|Schocken : Nextbook,|
|      Schocken Books|
|Schocken Books : ...|
|Schocken Books : ...|
|Schocken Books ; ...|
|Schocken Books ; ...|
|     Schocken Books,|
|Schocken Books, O...|
|Schocken Publishi...|
|           Schocken,|
|Schoenberg, Arnol...|
|Schoharie County ...|
|            Schoken,|
|      Scholar Press,|
|Scholar's Facsimi...|
|     Scholarly Press|
|    Scholarly Press,|
|Scholarly Resourc...|
|Scholarly Resources,|
+--------------------+
only showing top 20 rows



## Subjects

- Has NULLs

In [12]:
get_basic_counts(spl_df, spl_df.Subjects)
check_nulls(spl_df, spl_df.Subjects, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.Subjects)
check_lengths(spl_df, spl_df.Subjects)

+---------------+------------------------+
|count(Subjects)|count(DISTINCT Subjects)|
+---------------+------------------------+
|        2621314|                  439996|
+---------------+------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|        65835|                 29323|
+-------------+----------------------+

+---------------+------------------------+
|count(Subjects)|count(DISTINCT Subjects)|
+---------------+------------------------+
|              0|                       0|
+---------------+------------------------+

+--------+----------------+
|Subjects|length(Subjects)|
+--------+----------------+
|    null|            null|
|    null|            null|
|    null|            null|
|    null|            null|
|    null|            null|
+--------+----------------+
only showing top 5 rows

+--------------------+----------------+
|            Subjects|length(Subjects)|
+--------------

## Item Type

- Has NULLs

In [13]:
get_basic_counts(spl_df, spl_df.ItemType)
check_nulls(spl_df, spl_df.ItemType, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.ItemType)
check_lengths(spl_df, spl_df.ItemType)

+---------------+------------------------+
|count(ItemType)|count(DISTINCT ItemType)|
+---------------+------------------------+
|        2686132|                    1876|
+---------------+------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|         1017|                   447|
+-------------+----------------------+

+---------------+------------------------+
|count(ItemType)|count(DISTINCT ItemType)|
+---------------+------------------------+
|              0|                       0|
+---------------+------------------------+

+--------+----------------+
|ItemType|length(ItemType)|
+--------+----------------+
|    null|            null|
|    null|            null|
|    null|            null|
|    null|            null|
|    null|            null|
+--------+----------------+
only showing top 5 rows

+--------------------+----------------+
|            ItemType|length(ItemType)|
+--------------

## Item Collection

- Has NULLs

In [14]:
get_basic_counts(spl_df, spl_df.ItemCollection)
check_nulls(spl_df, spl_df.ItemCollection, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.ItemCollection)
check_lengths(spl_df, spl_df.ItemCollection)

+---------------------+------------------------------+
|count(ItemCollection)|count(DISTINCT ItemCollection)|
+---------------------+------------------------------+
|              2686323|                          1150|
+---------------------+------------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|          826|                   386|
+-------------+----------------------+

+---------------------+------------------------------+
|count(ItemCollection)|count(DISTINCT ItemCollection)|
+---------------------+------------------------------+
|                    0|                             0|
+---------------------+------------------------------+

+--------------+----------------------+
|ItemCollection|length(ItemCollection)|
+--------------+----------------------+
|          null|                  null|
|          null|                  null|
|          null|                  null|
|          n

## Floating Item

- Has NULLs
- What is a floating item?

In [15]:
get_basic_counts(spl_df, spl_df.FloatingItem)
check_nulls(spl_df, spl_df.FloatingItem, spl_df.BibNum)
check_empty_strings(spl_df, spl_df.FloatingItem)
check_lengths(spl_df, spl_df.FloatingItem)

+-------------------+----------------------------+
|count(FloatingItem)|count(DISTINCT FloatingItem)|
+-------------------+----------------------------+
|            2686366|                         605|
+-------------------+----------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|          783|                   374|
+-------------+----------------------+

+-------------------+----------------------------+
|count(FloatingItem)|count(DISTINCT FloatingItem)|
+-------------------+----------------------------+
|                  0|                           0|
+-------------------+----------------------------+

+------------+--------------------+
|FloatingItem|length(FloatingItem)|
+------------+--------------------+
|        null|                null|
|        null|                null|
|        null|                null|
|        null|                null|
|        null|                null|
+---

## Report Date

- Has NULLs
- Only two reporting dates. I'm assuming this is when the report was generated. Though why would there be two dates?
  - Check distribution of dates


In [16]:
get_basic_counts(spl_df, spl_df.ReportDateTS)
check_nulls(spl_df, spl_df.ReportDateTS, spl_df.BibNum)
basic_stats(spl_df, spl_df.ReportDateTS)

+-------------------+----------------------------+
|count(ReportDateTS)|count(DISTINCT ReportDateTS)|
+-------------------+----------------------------+
|            2679824|                           2|
+-------------------+----------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|         7325|                  2487|
+-------------+----------------------+

+-------------------+--------------------+-------------------+-------------------+
|count(ReportDateTS)|   avg(ReportDateTS)|  min(ReportDateTS)|  max(ReportDateTS)|
+-------------------+--------------------+-------------------+-------------------+
|            2679824|1.5055450984410915E9|2017-09-01 00:00:00|2017-10-01 00:00:00|
+-------------------+--------------------+-------------------+-------------------+



## ItemCount

- Has NULLs
- Should convert this to integer

In [17]:
get_basic_counts(spl_df, spl_df.ItemCount)
check_nulls(spl_df, spl_df.ItemCount, spl_df.BibNum)

+----------------+-------------------------+
|count(ItemCount)|count(DISTINCT ItemCount)|
+----------------+-------------------------+
|         2686403|                      356|
+----------------+-------------------------+

+-------------+----------------------+
|count(BibNum)|count(DISTINCT BibNum)|
+-------------+----------------------+
|          746|                   355|
+-------------+----------------------+

